# Student Attendance Portal Using SupaBase

In [8]:
!pip install supabase python-dotenv pandas pytz

In [9]:
import os
from datetime import datetime
import pytz # for timezone handling
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv

### Load Environmental Variables and Create Supabase Client

In [10]:
import os
from supabase import create_client, Client

#Read the credentials from .env file
load_dotenv()

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [11]:
#Helper function to get current time in EST timezone
def get_current_est_time():
    est = pytz.timezone('America/New_York')
    return datetime.now(est)

print("Current EST Time:", get_current_est_time())

Current EST Time: 2026-01-28 19:36:53.515957-05:00


## Create Classroom in Supabase
##### Key features: 1. Prevents duplicate class creation; 2. Adds default code and attendance limit;  3. Shows database insertion

In [12]:
def create_classroom(class_name, code="1234", daily_limit=25):
    existing=(supabase.table("classroom_settings").select("*").eq("class_name", class_name).execute().data)
    if existing:
        return f"Classroom with name {class_name} already exists."
    
    supabase.table("classroom_settings").insert({
        "class_name": class_name,
        "code": code,
        "daily_limit": daily_limit,
        "is_open" : True
    }).execute()
    return f"Classroom {class_name} created successfully."

create_classroom("FE524", "PromptEng", 26)


'Classroom with name FE524 already exists.'

## Add Students to Roll Map
##### Key features: 1. Prevents duplicate roll numbers; 2. Writes to roll_map table

In [13]:
def add_students(class_name, roll_number, name):
    existing = (supabase.table("roll_map").select("*").eq("roll_number", roll_number).eq("class_name", class_name).execute().data)
    if existing:
        return f"Student with roll number {roll_number} already exists in class {class_name}."
    
    supabase.table("roll_map").insert({
        "class_name": class_name,
        "roll_number": roll_number,
        "name": name
    }).execute()
    return f"Student {name} with roll number {roll_number} added to class {class_name}."    

add_students("CS556", "004", "Nisha")

'Student with roll number 004 already exists in class CS556.'

## Mark Attendance
##### Key features: 1. Checks correct code; 2. Roll number exists; 3. Attendance not marked already; 4. Daily limit not exceeded

In [14]:
def mark_attendance(class_name, roll_number, code):
    today = get_current_est_time().date().isoformat()

    # 1️⃣ Validate class + code safely
    settings_resp = (
        supabase.table("classroom_settings")
        .select("daily_limit")
        .eq("class_name", class_name)
        .eq("code", code)
        .execute()
    )

    if not settings_resp.data:
        return "Invalid class name or code."

    daily_limit = settings_resp.data[0]["daily_limit"]

    # 2️⃣ Prevent duplicate attendance
    existing = (
        supabase.table("attendance")
        .select("roll_number")
        .eq("class_name", class_name)
        .eq("roll_number", roll_number)
        .eq("date", today)
        .execute()
        .data
    )

    if existing:
        return f"Attendance already marked for roll number {roll_number} in class {class_name} today."

    # 3️⃣ Enforce daily attendance limit
    count_today = (
        supabase.table("attendance")
        .select("roll_number", count="exact")
        .eq("class_name", class_name)
        .eq("date", today)
        .execute()
        .count
    )

    if count_today >= daily_limit:
        return f"Daily attendance limit reached for class {class_name}."

    # 4️⃣ Validate student roll number
    student_resp = (
        supabase.table("roll_map")
        .select("name")
        .eq("class_name", class_name)
        .eq("roll_number", roll_number)
        .execute()
    )

    if not student_resp.data:
        return f"Roll number {roll_number} not found in class {class_name}."

    name = student_resp.data[0]["name"]

    # 5️⃣ Insert attendance
    supabase.table("attendance").insert({
        "class_name": class_name,
        "roll_number": roll_number,
        "name": name,
        "date": today
    }).execute()

    return f"Attendance marked for {name} (Roll No: {roll_number}) in class {class_name}."


print(mark_attendance("CS559", "001", "FundaML"))
print(mark_attendance("CS556", "004", "MathML"))


Attendance already marked for roll number 001 in class CS559 today.
Attendance already marked for roll number 004 in class CS556 today.


## Show Attendance Matrix
##### Key features: 1. Rows -> Students; 2. Columns -> dates; 3. Values -> P/A; 4. Basic Analytics

In [ ]:
def attendance_matrix(class_name):
    records =(
        supabase.table("attendance")
        .select("*")
        .eq("class_name", class_name)
        .order("date", desc=True)
        .execute()
        .data
    )
    
    if not records:
        return f"No attendance records found for class {class_name}."   
    
    df = pd.DataFrame(records)
    df["status"] ="P"
    
    pivot_df = df.pivot_table(index=["roll_number", "name"], columns="date", values="status", fill_value="A", aggfunc='first').reset_index()
    
    return pivot_df
